## 운동 분류

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from pytorchtools import EarlyStopping
from temporalize import create_dataset
from gcn_source import PoseGCN
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import pickle

## data 

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, video_folder, transform=None):
        """
        :param video_folder: 영상 데이터가 저장된 폴더 경로
        :param transform: 영상에 적용할 변환 (예: 크기 조정, 텐서 변환 등)
        """
        self.video_folder = video_folder
        self.video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]  # .mp4 확장자를 가진 파일만 선택
        self.transform = transform

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_path = os.path.join(self.video_folder, self.video_files[idx])
        
        # 영상 파일 불러오기 (예시에서는 첫 번째 프레임만 불러오는 예시)
        video = Image.open(video_path)
        
        # 변환 적용 (필요한 경우)
        if self.transform:
            video = self.transform(video)
        
        # 이 예시에서는 영상 데이터와 레이블이 함께 있다고 가정하고, 레이블은 0으로 설정
        label = 0  # 레이블을 설정하는 방식은 데이터셋에 맞게 수정하세요.
        
        return video, label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화 (예: ImageNet 평균값)
])


In [ ]:
# 영상 데이터가 저장된 폴더 경로
video_folder = 'data'

# 데이터셋 객체 생성
dataset = VideoDataset(video_folder, transform=transform)

# 데이터 로더 객체 생성 (배치 크기 32, 셔플 활성화)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# 데이터 로더를 사용하여 배치 단위로 데이터 확인
for videos, labels in data_loader:
    print(videos.shape, labels.shape)
    break  # 한 배치만 출력


In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model = PoseGCN().to(device)
optimizer = optim.RAdam(model.parameters(),lr = 1e-3)
criterion = nn.MSELoss()
scheduler = CosineAnnealingLR(optimizer, T_max=25, eta_min=0.0001)
def saveModel():
    torch.save(model.state_dict(), f'model/classifying.pt')
loss_ = []
valoss_ = []
logger = {"train_loss": list(),
          "validation_loss": list(),

          }



In [ ]:
def training(epochs) :

    for epoch in range(epochs):
        running_train_loss = 0.0
        running_vall_loss = 0.0
        total = 0
        model.train()
        for data in train_x:
            inputs = data
            outputs = data

            optimizer.zero_grad()  # zero the parameter gradients
            encoded, predicted_outputs = model((inputs))  # predict output from the model
            train_loss = criterion(predicted_outputs, outputs)# calculate loss for the predicted output

            train_loss.backward()  # backpropagate the loss
            optimizer.step()  # adjust parameters based on the calculated gradients
            running_train_loss += train_loss.item()  # track the loss value

        loss_.append(running_train_loss / n)
        with torch.no_grad():
            model.eval()
            for data in valid_x:
                inputs = data
                outputs = data

                encoded, predicted_outputs = model((inputs))
                val_loss = criterion(predicted_outputs, outputs)

                # The label with the highest value will be our prediction
                _, predicted = torch.max(predicted_outputs, 1)
                running_vall_loss += val_loss.item()
                total += outputs.size(0)
                val_loss_value = running_vall_loss / len(valid_x)


        valoss_.append(val_loss_value)

        avgtrainloss = np.mean(loss_)
        avgvalidloss = np.mean(valoss_)
        print('epoch', epoch + 1)
        print(f'train loss : {avgtrainloss}, validation loss : {avgvalidloss}')
        early_stopping(avgvalidloss, model)
        if early_stopping.early_stop:  # 조건 만족 시 조기 종료
            break
    saveModel()